In [768]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from random import choice
from queue import PriorityQueue

In [769]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets.toarray()

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [770]:
PROBLEM_SIZE = [100, 1000, 5000]
DENSITY = [.3, .7]

In [771]:
def tweak(state, problem_size):
    new_state = copy(state)
    index = randint(0,  problem_size - 1)
    new_state[index] = not new_state[index]
    return new_state

def fitness(state, problem_size, sets):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_size)]),
        )
    )
    return valid, -cost

# Hill Climbing with Termination

In [ ]:
def hill_climbing_with_termination(fitness_func, state_current, no_improvement_iterations, problem_size, sets):
    counter = 0
    fitness_call = 0
    no_improvements = 0
    iterations = 0
    current_best_state_fitness = fitness_func(state_current, problem_size, sets)
    for step in range(10000):
        iterations += 1
        if no_improvements >= no_improvement_iterations:
            break
        new_state = tweak(state_current, problem_size)
        new_state_fitness = fitness_func(new_state, problem_size, sets)
        fitness_call += 1
        if  new_state_fitness > current_best_state_fitness:
            counter += 1
            no_improvements = 0
            state_current = new_state
            current_best_state_fitness = new_state_fitness
        else:
            no_improvements += 1
    return state_current, current_best_state_fitness, fitness_call
    
list_best_solutions = []
for size in PROBLEM_SIZE:
    for density in DENSITY:
        best_solution = None
        for max_iter in range(size//10, size+1, size//10):
            SETS = make_set_covering_problem(size, size, density)
            current_state = [choice([False, False, False, False, False, False]) for _ in range(size)]
            state, fitness_state, fitness_call = hill_climbing_with_termination(fitness, current_state, max_iter, size, SETS)
            if best_solution is None or (fitness_state > best_solution[0]) or (fitness_state == best_solution[0] and fitness_call < best_solution[1]):
                best_solution = (fitness_state, fitness_call, max_iter, size, density)
        list_best_solutions.append(best_solution)

for index, sol in enumerate(list_best_solutions):
    print(f"Hill climbing with termination with PROBLEM_SIZE: {sol[3]} and DENSITY: {sol[4]}")
    print(f"FITNESS_STATE: {sol[0]}\nFITNESS CALL: {sol[1]}\nMAX_ITERATIONS: {sol[2]}")


# Hill Climbing With Random Restart

In [ ]:
def hill_climbing_with_random_restart(fitness_func, state_current, no_improvement_iterations, problem_size, sets, prob):
    counter = 0
    fitness_call = 0
    no_improvements = 0
    iterations = 0
    current_best_state_fitness = fitness_func(state_current, problem_size, sets)
    current_best_state = copy(state_current)
    for step in range(10000):
        iterations += 1
        if no_improvements >= no_improvement_iterations:
            current_best_state = copy(state_current)
            state_current = [random() < prob for _ in range(size)]
        new_state = tweak(state_current, problem_size)
        new_state_fitness = fitness_func(new_state, problem_size, sets)
        fitness_call += 1
        if  new_state_fitness > current_best_state_fitness:
            counter += 1
            no_improvements = 0
            state_current = new_state
            current_best_state = copy(state_current)
            current_best_state_fitness = new_state_fitness
        else:
            no_improvements += 1
    return current_best_state, current_best_state_fitness, fitness_call


list_best_solutions = []
for size in PROBLEM_SIZE:
    for density in DENSITY:
        best_solution = None
        for max_iter in range(size//10, size+1, size//10):
            for prob in [0.1, 0.2, 0.3]:
                SETS = make_set_covering_problem(size, size, density)
                current_state = [choice([False, False, False, False, False, False]) for _ in range(size)]
                state, fitness_state, fitness_call = hill_climbing_with_random_restart(fitness, current_state, max_iter, size, SETS, prob)
                if best_solution is None or (fitness_state > best_solution[0]) or (fitness_state == best_solution[0] and fitness_call < best_solution[1]):
                    best_solution = (fitness_state, fitness_call, max_iter, size, density, prob)
        list_best_solutions.append(best_solution)

for index, sol in enumerate(list_best_solutions):
    print(f"Hill climbing with random restart with PROBLEM_SIZE: {sol[3]} and DENSITY: {sol[4]}")
    print(f"FITNESS_STATE: {sol[0]}\nFITNESS CALL: {sol[1]}\nMAX_ITERATIONS: {sol[2]}\nPROB FOR RESTART: {sol[5]}")

# Hill climbing with steepest step

In [ ]:
def tweak_index(state, index):
    new_state = copy(state)
    new_state[index] = not state[index]
    return new_state

def steepest_step(fitness, state, size, sets):
    new_state = tweak_index(state, 0)
    new_state_fitness = fitness(new_state, size, sets)
    for index in range(0, size):
            temp_state = tweak_index(state, index)
            temp_state_fitness = fitness(temp_state, size, sets)
            if temp_state_fitness > new_state_fitness:
                new_state = temp_state
                new_state_fitness = temp_state_fitness
    return new_state, new_state_fitness

def hill_climbing_steepest_step(fitness_func, state_current, no_improvement_iterations, problem_size, sets, prob):
    counter = 0
    fitness_call = 0
    no_improvements = 0
    current_best_state_fitness = fitness_func(state_current, problem_size, sets)
    for step in range(1000):
        if no_improvements >= no_improvement_iterations:
            break
        a = random() < prob
        if not a:
            new_state, new_state_fitness = steepest_step(fitness, state_current, problem_size, sets)
            fitness_call += problem_size
        else:
            new_state = tweak(state_current, problem_size)
            new_state_fitness = fitness_func(new_state, problem_size, sets)
            fitness_call += 1
        if new_state_fitness > current_best_state_fitness:
            counter += 1
            no_improvements = 0
            state_current = new_state
            current_best_state = copy(state_current)
            current_best_state_fitness = new_state_fitness
        else:
            no_improvements += 1
    return current_best_state, current_best_state_fitness, fitness_call


list_best_solutions = []
for size in PROBLEM_SIZE:
    for density in DENSITY:
        best_solution = None
        for max_iter in range(size//10, size+1, size//10):
            SETS = make_set_covering_problem(size, size, density)
            current_state = [choice([False, False, False, False, False, False]) for _ in range(size)]
            state, fitness_state, fitness_call = hill_climbing_steepest_step(fitness, current_state, 500, size, SETS, prob)
            if best_solution is None or (fitness_state > best_solution[0]) or (fitness_state == best_solution[0] and fitness_call < best_solution[1]):                  
                best_solution = (fitness_state, fitness_call, max_iter, size, density, prob)       
        list_best_solutions.append(best_solution)

for index, sol in enumerate(list_best_solutions):
    print(f"Hill climbing with steepest step with PROBLEM_SIZE: {sol[3]} and DENSITY: {sol[4]}")
    print(f"FITNESS_STATE: {sol[0]}\nFITNESS CALL: {sol[1]}\nMAX_ITERATIONS: {sol[2]} PROB: {sol[5]}")